## tronsformaciones para recomendacion

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer



In [8]:
df_movies = pd.read_csv('ETL\movies_ETL.csv')

In [9]:
df_movies_cluster = df_movies.copy()


In [10]:
df_movies_cluster.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,1995,12.451801
1,NaN,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035
2,Grumpy Old Men Collection,0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.712900,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000
3,NaN,16000000,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760
4,Father of the Bride Collection,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000


In [11]:
#### Limpieza de datos para hacer mas simle las cadenas de texto, en el caso que estas tengan caracteres especiales
#%pip install unidecode
import re
from unidecode import unidecode
def limpiar_texto(texto):
    texto_sin_saltos = texto.replace("\n", " ")
    texto_minusculas = texto_sin_saltos.lower()
    texto_sin_tildes = unidecode(texto_minusculas)
    patron = r'@[\w]+|#\w+|[!,".]|(\b[^\w\s]\b)|\bhttps?\S+\b'
    texto_limpio = re.sub(patron, "", texto_sin_tildes)
    return texto_limpio

In [12]:
df_movies_cluster.dtypes



belongs_to_collection     object
budget                     int64
genres                    object
id                         int64
original_language         object
overview                  object
popularity               float64
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
vote_average             float64
vote_count               float64
release_year               int64
return                   float64
dtype: object

In [14]:
#dado que la columna overview se muestra en tipo object, se aplica la siguiente tranformacion para convertirlo en str
df_movies_cluster['overview'] = df_movies_cluster['overview'].astype(str)


In [15]:
#al texto almacenado en la columna overview se le aplica la funcion de limpieza creada anteriormente y se guarda en una nueva columna
df_movies_cluster['overview_limpio'] = df_movies_cluster['overview'].apply(limpiar_texto)

In [16]:
#verificamos
df_movies_cluster.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,overview_limpio
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,...,81.0,English,Released,NaN,Toy Story,7.7,5415.0,1995,12.451801,led by woody andys toys live happily in his ro...


In [17]:
### Tokenizar y quitar stopwords
## %pip install nltk
import nltk
nltk.download('stopwords') #se descarga un conjunto de palabrasque se consideran comunes y que generalmente no aportan mucho significado
from nltk.corpus import stopwords #se limitan las palabras al ingles 
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\makaw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# importa el módulo RegexpTokenizer de la biblioteca NLTK

from nltk.tokenize import RegexpTokenizer
regexp = RegexpTokenizer('\w+')# Creamos un tokenizador de expresiones regulares para dividir el texto en palabras


In [19]:
#aplicams tokenizacion y verificamos
df_movies_cluster['overview_token'] = df_movies_cluster['overview_limpio'].apply(regexp.tokenize)
df_movies_cluster.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,overview_limpio,overview_token
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,...,English,Released,NaN,Toy Story,7.7,5415.0,1995,12.451801,led by woody andys toys live happily in his ro...,"[led, by, woody, andys, toys, live, happily, i..."
1,NaN,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,...,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035,when siblings judy and peter discover an encha...,"[when, siblings, judy, and, peter, discover, a..."
2,Grumpy Old Men Collection,0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.712900,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,...,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000,a family wedding reignites the ancient feud be...,"[a, family, wedding, reignites, the, ancient, ..."
3,NaN,16000000,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,...,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760,cheated on mistreated and stepped on the women...,"[cheated, on, mistreated, and, stepped, on, th..."
4,Father of the Bride Collection,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,...,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000,just when george banks has recovered from his ...,"[just, when, george, banks, has, recovered, fr..."


In [20]:
#eliminamos palabras comunes y poco relevantes en el lenguaje
df_movies_cluster['overview_token_stopwords'] = df_movies_cluster['overview_token'].apply(lambda x: [item for item in x if item not in stopwords])
df_movies_cluster.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,status,tagline,title,vote_average,vote_count,release_year,return,overview_limpio,overview_token,overview_token_stopwords
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,...,Released,NaN,Toy Story,7.7,5415.0,1995,12.451801,led by woody andys toys live happily in his ro...,"[led, by, woody, andys, toys, live, happily, i...","[led, woody, andys, toys, live, happily, room,..."
1,NaN,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035,when siblings judy and peter discover an encha...,"[when, siblings, judy, and, peter, discover, a...","[siblings, judy, peter, discover, enchanted, b..."
2,Grumpy Old Men Collection,0,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,11.712900,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000,a family wedding reignites the ancient feud be...,"[a, family, wedding, reignites, the, ancient, ...","[family, wedding, reignites, ancient, feud, ne..."
3,NaN,16000000,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760,cheated on mistreated and stepped on the women...,"[cheated, on, mistreated, and, stepped, on, th...","[cheated, mistreated, stepped, women, holding,..."
4,Father of the Bride Collection,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000,just when george banks has recovered from his ...,"[just, when, george, banks, has, recovered, fr...","[george, banks, recovered, daughters, wedding,..."


In [21]:
# Importamos la biblioteca NLTK para la lematización
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


# Ejecutar (en la terminal) el siguiente comando para descargar el modelo de lenguaje en inglés de Spacy
#python -m spacy download en_core_web_sm
# Cargamos el modelo de lenguaje en inglés de Spacy
import spacy
#%pip install spacy
nlp = spacy.load("en_core_web_sm")
#nltk.download('wordnet')

# Creamos una instancia del lematizador de WordNet
lemmatizer = WordNetLemmatizer()

# Definimos una función para obtener la categoría gramatical según el modelo de Spacy
def get_wordnet_pos(text):
  pos = nlp(text)[0].pos_
  if pos == 'VERB':
    return 'v'
  elif pos == 'ADJ':
    return 'a'
  elif pos == 'ADV':
    return 'r'
  elif pos == 'NOUN':
    return 'n'
  else:
    return 'n'
  
# Creamos una función para realizar la lematización
lemmatizer = WordNetLemmatizer()
def lematizacion(text):
  return lemmatizer.lemmatize(text, get_wordnet_pos(text)) 



In [ ]:
# se aplica la funcion de lematizacion y se guarda en una nueva columna
df_movies_cluster['overview_lemmatization'] = df_movies_cluster['overview_token_stopwords'].apply(lambda x: [lematizacion(item) for item in x ])

#NOTA: esta línea de código demora más de 3 horas en ejecutarse,  por lo que recurrí a Google colab para ejecutar el código hasta aquí y guardar los datos en un archivo llamado df_movies_cluster.csv

In [ ]:
#cargamos el archivo resultante de la lemaiacion en Google colab
df_movies_cluster = pd.read_csv('ETL\df_movies_cluster.csv')

In [ ]:
type(df_movies_cluster.loc[0, 'overview_lemmatization'])

list

In [ ]:
#devido a que los datos de la lematizacion quedaron en formato lista, se utilizara ast.literal_eval para obtener su valor literal
import ast

df_movies_cluster['overview_lemmatization'] = df_movies_cluster['overview_lemmatization'].apply(ast.literal_eval)

In [ ]:
# unimos el texto que esta separado por comas
df_movies_cluster['overview_lemmatization_completo'] = df_movies_cluster['overview_lemmatization'].apply(lambda x: ' '.join([item for item in x]))

In [ ]:
df_movies_cluster.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,title,vote_average,vote_count,release_year,return,overview_limpio,overview_token,overview_token_stopwords,overview_lemmatization,overview_lemmatization_completo
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,...,Toy Story,7.7,5415.0,1995,12.451801,led by woody andys toys live happily in his ro...,"['led', 'by', 'woody', 'andys', 'toys', 'live'...","['led', 'woody', 'andys', 'toys', 'live', 'hap...","[lead, woody, andys, toy, live, happily, room,...",lead woody andys toy live happily room andys b...


In [ ]:
#relenamos los datos nulos con un espacio. no los borramos para conservar tituos de peliculas
df_movies_cluster['overview_lemmatization_completo'] = df_movies_cluster['overview_lemmatization_completo'].fillna("")

In [ ]:
#se guardan los cambios en el mismo archivo csv

df_movies_cluster.to_csv('ETL\df_movies_cluster.csv', index=False)